In [1]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.5.1-py3-none-any.whl size=14951 sha256=3ac30f424cc2fd1b134f436415e7194524b97733188ba6e281f9423a186cd3cb
  Stored in directory: /tmp/pip-ephem-wheel-cache-zh4li2mh/wheels/3d/ec/b0/a96d1d126183f98570a785e6bf8789fca559853a9260e928e1
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [ ]:
#To download the dataset from shared link (hardly ever works...)
!curl -L -o 'NLP_dataset.zip' 'https://drive.google.com/u/0/uc?id=1_UqE0UPcIEeotujyY5Doim7k02hkD2ix&export=download&confirm=t'
# import gdown
# gdown.download(
#     f"https://drive.google.com/uc?export=download&confirm=pbef&id=1_UqE0UPcIEeotujyY5Doim7k02hkD2ix",
#     "NLP_dataset.zip"
# )
!mkdir dataset
!unzip NLP_dataset.zip -d dataset

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 3699M  100 3699M    0     0   162M      0  0:00:22  0:00:22 --:--:--  178M
mkdir: cannot create directory ‘dataset’: File exists
Archive:  NLP_dataset.zip
replace dataset/cleaned_dataset.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [2]:
#To get the dataset from drive

# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

!mkdir dataset
!unzip gdrive/MyDrive/NLP_dataset.zip -d dataset

Streaming output truncated to the last 5000 lines.
  inflating: dataset/images/000007602.jpg  
  inflating: dataset/images/000007603.jpg  
  inflating: dataset/images/000007604.jpg  
  inflating: dataset/images/000007605.jpg  
  inflating: dataset/images/000007606.jpg  
  inflating: dataset/images/000007607.jpg  
  inflating: dataset/images/000007608.jpg  
  inflating: dataset/images/000007609.jpg  
  inflating: dataset/images/000007610.jpg  
  inflating: dataset/images/000007611.jpg  
  inflating: dataset/images/000007612.jpg  
  inflating: dataset/images/000007613.jpg  
  inflating: dataset/images/000007614.jpg  
  inflating: dataset/images/000007615.jpg  
  inflating: dataset/images/000007616.jpg  
  inflating: dataset/images/000007617.jpg  
  inflating: dataset/images/000007618.jpg  
  inflating: dataset/images/000007619.jpg  
  inflating: dataset/images/000007620.jpg  
  inflating: dataset/images/000007621.jpg  
  inflating: dataset/images/000007622.jpg  
  inflating: dataset/imag

# Imports

In [3]:
import json
import random
import tensorflow as tf
from tqdm import tqdm
import numpy as np
from tensorflow import keras

# Preprocess Dataset

In [4]:
with open("dataset/cleaned_dataset.json", 'r') as f:
    dataset_json = json.load(f)

dataset_len = len(dataset_json)
dataset_ids = [*range(dataset_len)]

#shuffle dataset
random.Random(3).shuffle(dataset_ids)

captions = [f"<start> {dataset_json[id]['caption']} <end>" for id in dataset_ids]
image_paths = ['dataset/images/{0:09d}.jpg'.format(id) for id in dataset_ids]

#take 80% as train set, and 20% as test
train_length = int(dataset_len * .8)
test_length = dataset_len - train_length

##Preprocess images

In [5]:
def load_image(image_path):
  img = tf.io.read_file(image_path)
  img = tf.io.decode_jpeg(img, channels=3)
  img = tf.keras.layers.Resizing(299, 299)(img)
  img = tf.keras.applications.inception_v3.preprocess_input(img)
  return img, image_path

def create_inceptionv3_model():
  inception_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
  return tf.keras.Model(inception_model.input, inception_model.layers[-1].output)

In [6]:
#get image features from inceptionv3 model's last layer
img_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
img_dataset = img_dataset.map(
  load_image, num_parallel_calls=tf.data.AUTOTUNE).batch(32)

feature_extraction_model = create_inceptionv3_model()
for img, path in tqdm(img_dataset):
  features = feature_extraction_model(img)
  image_features = tf.reshape(features, (features.shape[0], -1, features.shape[3]))
  
  for bf, p in zip(image_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())

87924736/87910968 [==============================] - 0s 0us/step


100%|██████████| 394/394 [03:22<00:00,  1.95it/s]


## Preprocess captions

In [7]:
def get_longest_caption_length(captions):
  return max([len(caption.split()) for caption in captions])

output_sequence_length = get_longest_caption_length(captions)
train_captions = captions[:train_length] 

# create a tokenizer and get vocabulary from train captions using TextVectorization
tokenizer = tf.keras.layers.TextVectorization(
  standardize=None, # we already preprocessed captions in the last phase
  max_tokens=5000,
  output_sequence_length=output_sequence_length)

caption_dataset = tf.data.Dataset.from_tensor_slices(train_captions)
tokenizer.adapt(caption_dataset)

#get word2index and index2word for mapping the words and indices
word_to_index = tf.keras.layers.StringLookup(
    mask_token="",
    vocabulary=tokenizer.get_vocabulary())
index_to_word = tf.keras.layers.StringLookup(
    mask_token="",
    vocabulary=tokenizer.get_vocabulary(),
    invert=True)

# Create dataset and set hyperparameters

In [8]:
BATCH_SIZE = 64
embedding_dim = 512
EPOCHS = 40

In [9]:
# get tokenized vectors
# def get_vectors(caps):
#   cap_dataset = tf.data.Dataset.from_tensor_slices(caps)
#   return cap_dataset.map(lambda x: tokenizer(x))

#image path and caption in string form will be turned to image features and a list of token indices for caption
def create_dataset_data(image_path, caption):
  cap_vec = tokenizer(caption)
  img_vec = np.load(image_path.decode('utf-8')+'.npy')
  return img_vec, cap_vec

In [10]:
train_image_paths = image_paths[:train_length]

train_dataset = tf.data.Dataset.from_tensor_slices((train_image_paths, train_captions))
train_dataset = train_dataset.map(lambda item1, item2: tf.numpy_function(
          create_dataset_data, [item1, item2], [tf.float32, tf.int64]),
          num_parallel_calls=tf.data.AUTOTUNE)

train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

# Create Model

In [20]:
class AdditiveAttention(tf.keras.Model):
  def __init__(self, units):
    super(AdditiveAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, features, hidden):
    hidden_with_time_axis = tf.expand_dims(hidden, 1)
    score = self.V(tf.nn.tanh(self.W1(features) +
                                         self.W2(hidden_with_time_axis)))
    normalized_score = tf.nn.softmax(score, axis=1)
    context_vector = normalized_score * features
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector

In [21]:
class DecoderWithAttention(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(DecoderWithAttention, self).__init__()
    self.attention = AdditiveAttention(512)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.lstm = tf.keras.layers.LSTM(512,
                                   return_sequences=True,
                                   return_state=True)
    self.lstm2 = tf.keras.layers.LSTM(512,
                                   return_sequences=True,
                                   return_state=True)
    self.fc = tf.keras.layers.Dense(400)
    self.norm = tf.keras.layers.BatchNormalization()
    self.out_fc = tf.keras.layers.Dense(vocab_size)

  
  def call(self, x, hidden, features):
    x = self.embedding(x)
    context_vector = self.attention(features, hidden)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
    x, _, _= self.lstm(x)
    output, new_hidden, _= self.lstm2(x)
    x = self.fc(output)
    x = tf.reshape(x, (-1, x.shape[2]))
    x = self.norm(x)
    x = self.out_fc(x)
    return x, new_hidden


In [22]:
class ImageCaptioningModel(keras.Model):
  def __init__(
        self, encoder, decoder, units
    ):
    super().__init__()
    self.units = units
    self.encoder = encoder
    self.decoder = decoder
    self.loss_tracker = keras.metrics.Mean(name="loss")
    self.acc_tracker = keras.metrics.Mean(name="accuracy")

  def calculate_loss(self, y_true, y_pred, mask):
    loss = self.loss(y_true, y_pred)
    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask
    mask_reduced = tf.reduce_sum(mask)
    if mask_reduced == 0:
      return 0
    return tf.reduce_sum(loss) / mask_reduced

  def calculate_accuracy(self, y_true, y_pred, mask):
    accuracy = tf.equal(y_true, tf.argmax(y_pred, axis=-1))
    accuracy = tf.math.logical_and(mask, accuracy)
    accuracy = tf.cast(accuracy, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    mask_reduced = tf.reduce_sum(mask)
    if mask_reduced == 0:
      return -1
    return tf.reduce_sum(accuracy) / mask_reduced

  @property
  def metrics(self): #not used anymore. wanted it to be used in model.fit() function
    return [self.loss_tracker, self.acc_tracker]

  def train_step(self, batch_data):
    batch_img, batch_seq = batch_data
    batch_loss = 0
    batch_acc = []

    batch_seq_size = batch_seq.get_shape().as_list()[0]
    hidden = tf.zeros((batch_seq_size, self.units))
    decoder_input = tf.expand_dims([word_to_index('<start>')] * batch_seq_size, 1)
    with tf.GradientTape() as tape:
      encoder_out = self.encoder(batch_img, training=True)
      for i in range(1, batch_seq.shape[1]):
        predictions, hidden = self.decoder(decoder_input, hidden, encoder_out)
        mask = tf.math.not_equal(batch_seq[:,i], 0)
        loss = self.calculate_loss(batch_seq[:,i], predictions, mask)
        batch_loss += loss
        acc = self.calculate_accuracy(batch_seq[:,i], predictions, mask)
        if acc != -1:
          batch_acc.append(acc) 

        # using teacher forcing
        decoder_input = tf.expand_dims(batch_seq[:, i], 1)
    
    total_loss = (batch_loss / int(batch_seq.shape[1]))
    total_acc = tf.reduce_mean(batch_acc)
    trainable_variables = self.encoder.trainable_variables + self.decoder.trainable_variables
    gradients = tape.gradient(batch_loss, trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, trainable_variables))

    self.loss_tracker.update_state(total_loss)
    self.acc_tracker.update_state(total_acc)

    return {"loss": self.loss_tracker.result(), "acc": self.acc_tracker.result()}

  def test_step(self, batch_data):
    batch_img, batch_seq = batch_data
    batch_loss = 0
    batch_acc = []

    encoder_out = self.encoder(batch_img)
    batch_seq_size = batch_seq.get_shape().as_list()[0]
    hidden = tf.zeros((batch_seq_size, self.units))
    decoder_input = tf.expand_dims([word_to_index('<start>')] * batch_seq_size, 1)

    for i in range(output_sequence_length):
      predictions, hidden= self.decoder(decoder_input, hidden, encoder_out)
      predicted_id = tf.random.categorical(predictions, 1)[:,0].numpy()
      mask = tf.math.not_equal(batch_seq[:,i], 0)
      loss = self.calculate_loss(batch_seq[:,i], predictions, mask)
      batch_loss += loss
      acc = self.calculate_accuracy(batch_seq[:,i], predictions, mask)
      if acc != -1:
        batch_acc.append(acc) 

      decoder_input = tf.expand_dims(predicted_id, 1)

    total_loss = (batch_loss / int(batch_seq.shape[1]))
    total_acc = tf.reduce_mean(batch_acc)
    self.loss_tracker.update_state(total_loss)
    self.acc_tracker.update_state(total_acc)

    return {"loss": self.loss_tracker.result(), "acc": self.acc_tracker.result()}


In [23]:
Image_encoder = tf.keras.models.Sequential()
Image_encoder.add(tf.keras.Input(shape=(None, None, 2048)))
Image_encoder.add(tf.keras.layers.Dense(embedding_dim,activation='relu'))
decoder = DecoderWithAttention(tokenizer.vocabulary_size(), embedding_dim)

In [24]:
cross_entropy = keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction="none"
)
optimizer = keras.optimizers.Adam(learning_rate=0.001)

checkpoint_path = "./gdrive/MyDrive/checkpoints/train"
ckpt = tf.train.Checkpoint(encoder=Image_encoder,
                           decoder=decoder,
                           optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

epoch_start = 0
if ckpt_manager.latest_checkpoint:
  epoch_start = int(ckpt_manager.latest_checkpoint.split('-')[-1])
  ckpt.restore(ckpt_manager.latest_checkpoint)


model = ImageCaptioningModel(Image_encoder, decoder,512)
model.compile(optimizer=optimizer, loss=cross_entropy)

In [25]:
for epoch in range(epoch_start, EPOCHS):
    total_loss = 0
    total_acc = 0
    iter = 1

    for (batch, (img_tensor, target)) in enumerate(train_dataset):
        loss_acc = model.train_step((img_tensor, target))
        total_loss += loss_acc["loss"]
        total_acc += loss_acc["acc"]
        iter+=1
        
        if batch % 50 == 0:
            print(f'Epoch {epoch+1} Batch {batch} Loss {loss_acc["loss"]:.4f} Acc {loss_acc["acc"]:.4f}')

    ckpt_manager.save()

    total_loss = total_loss / iter
    total_acc = total_acc / iter
    print(f'Epoch {epoch+1} Loss {total_loss} Accuracy {total_acc}')

Epoch 34 Batch 0 Loss 0.9521 Acc 0.6165
Epoch 34 Batch 50 Loss 0.9279 Acc 0.6767
Epoch 34 Batch 100 Loss 0.9203 Acc 0.6806
Epoch 34 Batch 150 Loss 0.9231 Acc 0.6830
Epoch 34 Loss 0.917005717754364 Accuracy 0.6708135604858398
Epoch 35 Batch 0 Loss 0.9209 Acc 0.6837
Epoch 35 Batch 50 Loss 0.9107 Acc 0.6858
Epoch 35 Batch 100 Loss 0.9051 Acc 0.6877
Epoch 35 Batch 150 Loss 0.9060 Acc 0.6887
Epoch 35 Loss 0.9034331440925598 Accuracy 0.6819859743118286
Epoch 36 Batch 0 Loss 0.9037 Acc 0.6893
Epoch 36 Batch 50 Loss 0.8917 Acc 0.6923
Epoch 36 Batch 100 Loss 0.8885 Acc 0.6934
Epoch 36 Batch 150 Loss 0.8870 Acc 0.6938
Epoch 36 Loss 0.8857061266899109 Accuracy 0.687986433506012
Epoch 37 Batch 0 Loss 0.8861 Acc 0.6942
Epoch 37 Batch 50 Loss 0.8765 Acc 0.6967
Epoch 37 Batch 100 Loss 0.8728 Acc 0.6975
Epoch 37 Batch 150 Loss 0.8693 Acc 0.6989
Epoch 37 Loss 0.8697188496589661 Accuracy 0.6925575733184814
Epoch 38 Batch 0 Loss 0.8678 Acc 0.6993
Epoch 38 Batch 50 Loss 0.8595 Acc 0.7016
Epoch 38 Batch 10

# Testing

In [26]:
test_image_paths = image_paths[train_length:]
test_captions = captions[train_length:]

test_dataset = tf.data.Dataset.from_tensor_slices((test_image_paths, test_captions))
test_dataset = test_dataset.map(lambda item1, item2: tf.numpy_function(
          create_dataset_data, [item1, item2], [tf.float32, tf.int64]),
          num_parallel_calls=tf.data.AUTOTUNE)

test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

In [27]:
total_loss = 0
total_acc = 0
iter = 1
for (batch, (img_tensor, target)) in enumerate(test_dataset):
  loss_acc = model.test_step((img_tensor, target))
  total_loss += loss_acc["loss"]
  total_acc += loss_acc["acc"]
  iter+=1
  if batch % 50 == 0:
    print(f'Batch {batch} Loss {loss_acc["loss"]:.4f} Acc {loss_acc["acc"]:.4f}')

total_loss = total_loss / iter
total_acc = total_acc / iter
print(f'Loss {total_loss} Accuracy {total_acc}')

Batch 0 Loss 0.8336 Acc 0.7091
Loss 0.9147889614105225 Accuracy 0.680516242980957
